In [ ]:
%matplotlib inline
import os
import matplotlib.pyplot as plt
import numpy as np
import rasterio
import firedrake
import icepack, icepack.plot, icepack.models

# Larsen Ice Shelf

This demo will involve using real data for the Larsen Ice Shelf in the Antarctic Peninsula.
So far as the actual modeling is concerned, most of what we'll need was in the previous demo.
What's new here are the functions for reading in meshes and gridded data sets from files.

The scripts I used to fetch and process all of this data are contained in [this repo](https://github.com/icepack/icepack-data).
To use it, clone the repository and run `make` in each of the directories `bedmap2/`, `measures-antarctica/`, and `meshes/larsen/`.
For this notebook, I'll assume that the absolute path of the data repository on your computer is stored in a variable `data_directory`.
In my case, I keep this in an environment variable called `ICEPACK_DATA`.
If you modify this notebook for your own uses, you can use whatever works for you to get the path right; the easiest way would probably be to hard-code `data_directory` to the right location on your computer.

In [ ]:
data_directory = os.environ['ICEPACK_DATA']
print(data_directory)

### Geometry

First we need to make a mesh.
I go about this by hand-drawing the domain boundary in [QGIS](https://www.qgis.org) on top of satellite imagery and whatever other gridded data sets I'm working with.
I always save these vector data in the [GeoJSON](http://geojson.org/) format rather than, say, an ESRI shapefile.
GeoJSON is human-readable and easy to keep in version control.
The GeoJSON files for the outline of Larsen are in the directory of `meshes/larsen/` of the [data repo](https://github.com/icepack/icepack-data).
That said, this is my own peculiar workflow and yours might differ.

The domain outline is then transformed into whatever file format your mesh generator of choice can read.
For this example, I used [gmsh](http://gmsh.info/).
Finally, we invoke the mesh generator to turn our description of the outline into a mesh of the interior of the domain.

Firedrake has built-in functions for reading meshes in a variety of formats.
The following code reads in the Larsen mesh and makes a plot of it so that we can see all the boundary IDs.
Boundary segments 1 and 2 correspond to the calving terminus; segment 3 borders the Gipps Ice Rise.

In [ ]:
mesh = firedrake.Mesh(os.path.join(data_directory, 'meshes', 'larsen', 'larsen.msh'))

fig, axes = icepack.plot.subplots()
axes.set_xlabel('meters')
axes.grid()
icepack.plot.triplot(mesh, axes=axes, linewidth=2)
plt.show(fig)

### Input data

Next, we have to load the input data.
We'll use ice thickness data from [Bedmap2](https://www.bas.ac.uk/project/bedmap-2/) and velocity data from [MEaSUREs](https://nsidc.org/data/measures/aiv).
The scripts in the `bedmap2` and `measures-antarctica` directories of the [data repo](https://github.com/icepack/icepack-data) fetch these data sets from their various hosts online.

First, we'll read in the thickness data, which are stored in the [GeoTIFF](https://en.wikipedia.org/wiki/GeoTIFF) format.
GeoTIFF is a common storage format for geospatial data; it adds georeferencing information on top of the TIFF file format, which is often used for lossless compression of images.
We'll use the [rasterio](https://rasterio.readthedocs.io/en/stable/) package, which has functions to open GeoTIFF and other file formats.
The function `rasterio.open` will give us an object representing the raster data set that we can then read from.

In [ ]:
bedmap2_filename = os.path.join(data_directory, 'bedmap2', 'bedmap2_tiff', 'bedmap2_thickness.tif')
thickness = rasterio.open(bedmap2_filename, 'r')

Opening the file doesn't immediately read all the data at once.
Instead, that occurs through calls to the `read` method of the data set, in our case the variable `thickness`.
The actual reading will occur entirely within the functions that interpolate the raster data to our computational mesh.
This two-step procedure involves a little extra code, but the nice part is that you can read only as small a chunk of the data as need be.
It would be awfully wasteful if you had to load up the thickness of all of Antarctica only to focus on a little piece like the Larsen Ice Shelf.

Before going on, let's make a plot of the thickness data.
Plotting all your data first is a good sanity check to make sure that it isn't, say, stored upside-down, or in a different coordinate system than you expected.
To visualize our data, we'll use the function `contourf` in the module `icepack.plot`, which works just like the equivalent matplotlib function.

In [ ]:
fig, axes = icepack.plot.subplots()
contours = icepack.plot.contourf(thickness, 40, axes=axes)
fig.colorbar(contours, label='meters')
plt.show(fig)

Next, we have to read in the velocity data.
The velocity data are stored in a [NetCDF](https://en.wikipedia.org/wiki/NetCDF) file.
Many fields can be stored in one NetCDF file; we only want the $x$- and $y$-components of the velocity.
To specify which field we're reading, we can prepend `netcdf:` to the beginning of the filename and append `:FIELD_NAME` to the string we pass to `rasterio.open`.
The field names for the velocity components in this data set are `VX` and `VY`.

In [ ]:
measures_filename = os.path.join(data_directory, 'measures-antarctica', 'antarctica_ice_velocity_450m_v2.nc')
vx = rasterio.open('netcdf:' + measures_filename + ':VX', 'r')
vy = rasterio.open('netcdf:' + measures_filename + ':VY', 'r')

You can see which variables are stored in a NetCDF file by running the shell command `ncinfo` on the file.

In [ ]:
!ncinfo "$measures_filename"

### Modeling

Having done all the leg work to make a mesh and get a good set of input data, the modeling itself should be fairly familiar from the last step.
We'll assume that the ice temperature is a uniform $-13^\circ$C.

One thing is substantially different from previous examples.
Before, we called the function `firedrake.SpatialCoordinate` to get some symbolic handles `x, y` for the mesh coordinates, and we created symbolic expressions to define the input data to our problem analytically.
When we work with real data, we instead use icepack's `GridData` object, which firedrake doesn't know how to interpolate.
The function `icepack.interpolate` works as a layer on top of the firedrake interpolate function and knows what to do with gridded data sets.

In [ ]:
degree = 2
Q = firedrake.FunctionSpace(mesh, family='CG', degree=degree)
V = firedrake.VectorFunctionSpace(mesh, family='CG', degree=degree)

h0 = icepack.interpolate(thickness, Q)
u0 = icepack.interpolate((vx, vy), V)

In [ ]:
fig, axes = icepack.plot.subplots()
streamlines = icepack.plot.streamplot(u0, precision=1000, density=2500, axes=axes)
fig.colorbar(streamlines, label='meters/year')
plt.show(fig)

In [ ]:
T = 260
A = firedrake.interpolate(firedrake.Constant(icepack.rate_factor(T)), Q)

ice_shelf = icepack.models.IceShelf()
opts = {'dirichlet_ids': [3, 4, 5, 6, 7, 8], 'tol': 1e-6}
u = ice_shelf.diagnostic_solve(u0=u0, h=h0, A=A, **opts)

In [ ]:
fig, axes = icepack.plot.subplots()
streamlines = icepack.plot.streamplot(u, precision=1000, density=2500, axes=axes)
fig.colorbar(streamlines, label='meters/year')
plt.show(fig)

We get a fairly reasonable approximation for the velocity even with a spatially homogeneous guess for the ice temperature.

In [ ]:
print(icepack.norm(u - u0) / icepack.norm(u0))

Ballpark estimate, the surface and basal mass balance of Larsen C are +30 and -30 cm/yr respectively, so we can take the total to be 0.
Let's simulate the evolution of the ice shelf for the next 10 years.
The code for this loop should be familiar from the previous example.

In [ ]:
a = firedrake.Function(Q)
h = h0.copy(deepcopy=True)

dt = 0.5
for n in range(int(10 / dt) + 1):
    h = ice_shelf.prognostic_solve(dt, h0=h, a=a, u=u, h_inflow=h0)
    u = ice_shelf.diagnostic_solve(u0=u, h=h, A=A, **opts)

In [ ]:
fig, axes = icepack.plot.subplots()
contours = icepack.plot.tricontourf(h, 96, axes=axes)
fig.colorbar(contours, label='meters')
plt.show(fig)

By plotting the difference between the modeled thickness after 10 years and the initial thickness, we can see the propagation of the rifts downstream.
This effect is best visualized with a diverging colormap that makes the 0-contour really obvious.

In [ ]:
δh = firedrake.Function(Q)
δh.assign(h - h0)

fig, axes = icepack.plot.subplots()
contours = icepack.plot.tricontourf(δh, axes=axes, cmap='RdBu',
                                    levels=np.linspace(-10, +10, 101), extend='both')
fig.colorbar(contours, label='meters')
plt.show(fig)

The oscillatory pattern makes it less than obvious whether the ice shelf gained or lost mass, so let's evaluate the integral of the thickness change to see.

In [ ]:
from firedrake import assemble, dx
print(assemble(δh * dx) / assemble(1 * dx(mesh)))

Seeing as the simulation ran for 10 years, this isn't a wildly unrealistic number.

### Conclusion

In the last demo, we showed how to simulate ice shelf flow using synthetic data.
Here we showed how to load in a generated mesh and observational data, and we used this same functionality to simulate a real ice shelf.

Many real data sets require some amount of preprocessing before they can be used for modeling.
For example, many velocity data sets have missing pixels or patches due to noise in the optical or radar imagery, and these missing points have to be filled in somehow.
The Bedmap2 thickness also contains processing artifacts that are visible as depressions running diagonally across the ice shelf.
These artifacts could be removed by using a low-pass filter on the gridded data, although this might also wash out some real features like the many rifts in the ice.

In order to run the simulation, we had to come up with a guess for the ice rheology.
The simple choice we made is quite far from the real value and in a subsequent demo we'll show how to estimate it from observational data.